# Tarea Semana 2

En esta semana el objetivo es trabajar con los algoritmos de aprendizaje automático Decision 
Tree y Random Forest.

¿es posible generar un modelo que pueda predecir el Genero de un juego en base a las ventas globales, las ventas en Japón y la 
plataforma?

- Para poder trabajar con estos modelos, será necesario factorizar las variables categóricas
- Separar el dataset en dos, train y test
- Ejecutar Random Forest
- Ejecutar Arboles de Decision 
- Generar las matrices de confusión de ambos modelos, para poder comparar la performance



In [1]:
import pandas as pd
import numpy as  np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as  np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier

## Recuperar DataSet

In [2]:
df = pd.read_csv("vgsales v3.csv", sep=';')
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


# Analizar y preparar DataSet

In [3]:
#Ver tipos de datos
df.dtypes

Rank              int64
Name             object
Platform         object
Year            float64
Genre            object
Publisher        object
NA_Sales        float64
EU_Sales        float64
JP_Sales        float64
Other_Sales     float64
Global_Sales    float64
dtype: object

In [4]:
# Ver valores nulos

df.isnull().sum()

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          5
JP_Sales          4
Other_Sales       4
Global_Sales      6
dtype: int64

## Manejar valores nulos

Dado que existen valores nulos correspondientes a las ventas de cada lugar debemos solucionarlo para que no afecten el resultado final.
La suma de las ventas de cada lugar, resultan en Global Sales.

### Global Sales

Se realiza la suma de los valores de las columnas NA_Sales, EU_Sales, JP_Sales, y Other Sales

In [5]:
#Filas donde existen valores nulos

df[df['Global_Sales'].isnull()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
230,231,Guitar Hero III: Legends of Rock,Wii,2007.0,Misc,Activision,3.06,1.12,0.00,0.44,NaN
231,232,The Legend of Zelda: A Link to the Past,SNES,1991.0,Action,Nintendo,2.42,0.91,1.15,0.13,NaN
232,233,The Legend of Zelda: The Wind Waker,GC,2002.0,Action,Nintendo,2.60,0.99,0.89,0.13,NaN
233,234,Mario & Sonic at the Olympic Winter Games,Wii,2009.0,Sports,Sega,1.89,1.99,0.22,0.48,NaN
234,235,Luigi's Mansion: Dark Moon,3DS,2013.0,Action,Nintendo,1.78,1.39,1.10,0.30,NaN
339,340,Tony Hawk's Pro Skater 4,PS2,2002.0,Sports,Activision,2.13,1.18,0.01,0.35,NaN


In [6]:
randGlobal_Sales = df["EU_Sales"] + df["NA_Sales"] + df["JP_Sales"] + df["Other_Sales"]

In [7]:
df.Global_Sales = df.Global_Sales.fillna(randGlobal_Sales)

In [8]:
df[df['Global_Sales'].isnull()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales


### EU_Sales

Se realiza la suma de NA_Sales, JP_Sales y Other_Sales, y luego se le resta a Global_Sales, con el fin de obtener EU_Sales

In [9]:
#Filas donde existen valores nulos

df[df['EU_Sales'].isnull()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
76,77,Super Mario Kart,SNES,1992.0,Racing,Nintendo,3.54,NaN,3.81,0.18,8.76
131,132,Kingdom Hearts,PS2,2002.0,Role-Playing,Sony Computer Entertainment,3.64,NaN,1.49,0.07,6.40
132,133,Pokémon Crystal Version,GB,2000.0,Role-Playing,Nintendo,2.55,NaN,1.29,0.99,6.39
133,134,Halo 3: ODST,X360,2009.0,Shooter,Microsoft Game Studios,4.34,NaN,0.06,0.61,6.36
317,318,Namco Museum Vol.1,PS,1995.0,Misc,Sony Computer Entertainment,2.12,NaN,0.22,0.06,3.84


In [10]:
df.loc[np.isnan(df["EU_Sales"]), "EU_Sales"] = (df["Global_Sales"]-df["Other_Sales"]-df["JP_Sales"]-df["NA_Sales"])

In [11]:
df[df['EU_Sales'].isnull()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales


### JP_Sales 

Se realiza la suma de NA_Sales, EU_Sales y Other_Sales, y luego se le resta a Global_Sales, con el fin de obtener JP_Sales

In [12]:
#Filas donde existen valores nulos

df[df['JP_Sales'].isnull()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
53,54,Super Mario 3D Land,3DS,2011.0,Platform,Nintendo,4.89,2.99,NaN,0.78,10.79
54,55,Gran Turismo 5,PS3,2010.0,Racing,Sony Computer Entertainment,2.96,4.88,NaN,2.12,10.77
55,56,Call of Duty: Modern Warfare 2,PS3,2009.0,Shooter,Activision,4.99,3.69,NaN,1.63,10.69
319,320,Mario & Luigi: Bowser's Inside Story,DS,2009.0,Role-Playing,Nintendo,2.26,0.48,NaN,0.27,3.83


In [13]:
df.loc[np.isnan(df["JP_Sales"]), "JP_Sales"] = (df["Global_Sales"]-df["Other_Sales"]-df["EU_Sales"]-df["NA_Sales"])

In [14]:
df[df['JP_Sales'].isnull()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales


### Other Sales

Se realiza la suma de NA_Sales, JP_Sales y EU_Sales, y luego se le resta a Global_Sales, con el fin de obtener Other_Sales

In [15]:
#Filas donde existen valores nulos

df[df['Other_Sales'].isnull()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
338,339,Friend Collection,DS,2009.0,Misc,Nintendo,0.00,0.00,3.67,NaN,3.67
357,358,Star Wars: Battlefront II,PS2,2005.0,Shooter,LucasArts,2.18,1.02,0.03,NaN,3.59
377,378,FIFA Soccer 2004,PS2,NaN,Sports,Electronic Arts,0.59,2.36,0.04,NaN,3.49
378,379,Star Wars Battlefront (2015),XOne,2015.0,Shooter,Electronic Arts,1.94,1.22,0.02,NaN,3.49


In [16]:
df.loc[np.isnan(df["Other_Sales"]), "Other_Sales"] = (df["Global_Sales"]-df["JP_Sales"]-df["EU_Sales"]-df["NA_Sales"])

In [17]:
df[df['Other_Sales'].isnull()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales


In [18]:
# Ver valores nulos

df.isnull().sum()

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

# Factorizar variables categóricas

Se asigna un número a cada categoría

## Plataforma

In [19]:
df.Platform.value_counts()

DS      2163
PS2     2161
PS3     1329
Wii     1325
X360    1265
PSP     1213
PS      1196
PC       960
XB       824
GBA      822
GC       556
3DS      509
PSV      413
PS4      336
N64      319
SNES     239
XOne     213
SAT      173
WiiU     143
2600     133
NES       98
GB        98
DC        52
GEN       27
NG        12
SCD        6
WS         6
3DO        3
TG16       2
GG         1
PCFX       1
Name: Platform, dtype: int64

In [20]:
from sklearn.preprocessing import OrdinalEncoder

# Creamos el codificador 
encoder = OrdinalEncoder(categories=[["DS",      
"PS2",    
"PS3",     
"Wii",    
"X360",   
"PSP",     
"PS",      
"PC",      
"XB",       
"GBA",     
"GC",      
"3DS",      
"PSV",      
"PS4",      
"N64",     
"SNES",     
"XOne",     
"SAT",      
"WiiU",     
"2600",     
"NES",       
"GB",        
"DC",        
"GEN",       
"NG",        
"SCD",        
"WS",         
"3DO",        
"TG16",       
"GG",         
"PCFX"]])

# Ajustamos el codificador
encoder.fit(df[["Platform"]])
df["Platform_Factorizada"] = encoder.transform(df[["Platform"]])

In [21]:
df[["Platform", "Platform_Factorizada"]].head(10)

,Platform,Platform_Factorizada
0,Wii,3.0
1,NES,20.0
2,Wii,3.0
3,Wii,3.0
4,GB,21.0
5,GB,21.0
6,DS,0.0
7,Wii,3.0
8,Wii,3.0
9,NES,20.0


Verificar que todas las plataformas tengan un número asignado.

In [22]:
df.isnull().sum()

Rank                      0
Name                      0
Platform                  0
Year                    271
Genre                     0
Publisher                58
NA_Sales                  0
EU_Sales                  0
JP_Sales                  0
Other_Sales               0
Global_Sales              0
Platform_Factorizada      0
dtype: int64

In [23]:
df.Platform_Factorizada.value_counts()

0.0     2163
1.0     2161
2.0     1329
3.0     1325
4.0     1265
5.0     1213
6.0     1196
7.0      960
8.0      824
9.0      822
10.0     556
11.0     509
12.0     413
13.0     336
14.0     319
15.0     239
16.0     213
17.0     173
18.0     143
19.0     133
20.0      98
21.0      98
22.0      52
23.0      27
24.0      12
25.0       6
26.0       6
27.0       3
28.0       2
29.0       1
30.0       1
Name: Platform_Factorizada, dtype: int64

## Género

In [24]:
df.Genre.value_counts()

Action          3316
Sports          2346
Misc            1739
Role-Playing    1488
Shooter         1310
Adventure       1286
Racing          1249
Platform         886
Simulation       867
Fighting         848
Strategy         681
Puzzle           582
Name: Genre, dtype: int64

In [25]:
# Creamos el codificador 
encoder = OrdinalEncoder(categories=[[ "Action",
"Sports",
"Misc",
"Role-Playing", 
"Shooter",
"Adventure",
"Racing",
"Platform",
"Simulation",
"Fighting",
"Strategy",
"Puzzle"  
]])

# Ajustamos el codificador
encoder.fit(df[["Genre"]])
df["Genre_Factorizado"] = encoder.transform(df[["Genre"]])

In [26]:
df[["Genre", "Genre_Factorizado"]].head(10)

,Genre,Genre_Factorizado
0,Sports,1.0
1,Platform,7.0
2,Racing,6.0
3,Sports,1.0
4,Role-Playing,3.0
5,Puzzle,11.0
6,Platform,7.0
7,Misc,2.0
8,Platform,7.0
9,Shooter,4.0


In [27]:
df.Genre_Factorizado.value_counts()

0.0     3316
1.0     2346
2.0     1739
3.0     1488
4.0     1310
5.0     1286
6.0     1249
7.0      886
8.0      867
9.0      848
10.0     681
11.0     582
Name: Genre_Factorizado, dtype: int64

In [28]:
df.isnull().sum()

Rank                      0
Name                      0
Platform                  0
Year                    271
Genre                     0
Publisher                58
NA_Sales                  0
EU_Sales                  0
JP_Sales                  0
Other_Sales               0
Global_Sales              0
Platform_Factorizada      0
Genre_Factorizado         0
dtype: int64

# Separar el dataset en dos, train y test (80% - 20%)

In [29]:
X = df[["Global_Sales","JP_Sales", "Platform_Factorizada"]]
y= df["Genre_Factorizado"]

In [30]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=1)

# Ejecutar Random Forest

In [31]:
#Entrenar
clf = RandomForestClassifier(random_state=1)
clf.fit(X_train,y_train)


RandomForestClassifier(random_state=1)

In [32]:
#Hacer predicción
y_pred=clf.predict(X_test)


## Evaluar Modelo

In [33]:

print(confusion_matrix(y_test,y_pred))

print(classification_report(y_test,y_pred))

print("Accuracy: ", accuracy_score(y_test, y_pred))


[[207 111  58  42  58  50  50  22  22  13  22  20]
 [122 126  47  28  26  15  37  14  11  22  12   8]
 [ 87  59  69  20  12  25  23   3  16   3   5  10]
 [ 80  33  21  65   8  25  13  10   7  11  17   5]
 [ 83  49  18  19  24  12  31   6   9   5  18   5]
 [ 60  17  34  18   4  75   9   5  12   4  13   7]
 [ 63  65  22  14  19   9  31   9   5  10   8   6]
 [ 33  26  19  13   9   6  18  16   5   8   3  12]
 [ 50  19  28  12   4  16   4   7  16   2  13   6]
 [ 37  29  13  22  12  16  12   6   1  13   1   5]
 [ 28  19  20   9   8  13   3   3   5   6  20   2]
 [ 20   8  16  13   3   5   4   5   9   4   7  10]]
              precision    recall  f1-score   support

         0.0       0.24      0.31      0.27       675
         1.0       0.22      0.27      0.24       468
         2.0       0.19      0.21      0.20       332
         3.0       0.24      0.22      0.23       295
         4.0       0.13      0.09      0.10       279
         5.0       0.28      0.29      0.29       258
        

# Ejecutar árboles de decisión

In [34]:
#Entrenar
classifier = DecisionTreeClassifier(random_state=1)
classifier.fit(X_train, y_train)


DecisionTreeClassifier(random_state=1)

In [35]:
#Hacer predicción

y_pred = classifier.predict(X_test)

## Evaluar Modelo

In [36]:

print(confusion_matrix(y_test, y_pred))
print("")
print(classification_report(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))


[[260 121  57  32  44  43  37  16  17  13  19  16]
 [159 125  42  29  20  20  32   9   9  12   8   3]
 [104  62  58  21  11  27  18   1  17   2   3   8]
 [ 95  40  26  50   7  26  11   7   7   8  14   4]
 [ 92  46  15  16  27  13  28   8   8   8  14   4]
 [ 68  23  37  15   3  71  10   3   9   2  10   7]
 [ 82  68  25  14  16   5  23   6   5   4   6   7]
 [ 40  31  19  11  10   5  14  17   5   4   3   9]
 [ 53  24  23  14   2  16   4   5  17   2  11   6]
 [ 54  32  10  18  12  16   6   2   1  11   1   4]
 [ 34  19  18  11   6  14   3   4   6   3  17   1]
 [ 22  12  16  11   2   6   2   3  11   4   6   9]]

              precision    recall  f1-score   support

         0.0       0.24      0.39      0.30       675
         1.0       0.21      0.27      0.23       468
         2.0       0.17      0.17      0.17       332
         3.0       0.21      0.17      0.19       295
         4.0       0.17      0.10      0.12       279
         5.0       0.27      0.28      0.27       258
       